[![Labellerr](https://storage.googleapis.com/labellerr-cdn/%200%20Labellerr%20template/notebook.webp)](https://www.labellerr.com)

# **Fine-Tune YOLO for Automated Product Counting**

---

[![labellerr](https://img.shields.io/badge/Labellerr-BLOG-black.svg)](https://www.labellerr.com/blog/<BLOG_NAME>)
[![Youtube](https://img.shields.io/badge/Labellerr-YouTube-b31b1b.svg)](https://www.youtube.com/@Labellerr)
[![Github](https://img.shields.io/badge/Labellerr-GitHub-green.svg)](https://github.com/Labellerr/Hands-On-Learning-in-Computer-Vision)
[![Scientific Paper](https://img.shields.io/badge/Official-Paper-blue.svg)](<PAPER LINK>)

### **Create Dataset and convert it in YOLO format**

In [ ]:
!git clone https://github.com/Labellerr/yolo_finetune_utils.git

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.bbox_converter import coco_to_yolo_converter

result = coco_to_yolo_converter(
            json_path=r'./dataset-2/train/annotations.json',
            images_dir=r'./dataset-2/train',
            output_dir='yolo_format-3',
            use_split=False
            )

In [ ]:
import ultralytics
ultralytics.checks()
from ultralytics import YOLO

In [ ]:
!yolo task=detect mode=train data="path/to/dataset.yaml" model="yolo11x.pt" epochs=400 imgsz=640 batch=20

In [ ]:
!yolo task=detect mode=track model="./runs/detect/train/weights/last.pt" source="./video/1.mp4" conf=0.25 save=True show_labels=False

---

## **Manual line input from user for Production Pipeline**

In [ ]:
# !pip install ultralytics opencv-python numpy matplotlib ipywidgets

import cv2
import numpy as np
from ultralytics import YOLO
from datetime import datetime

print("✅ All packages imported successfully!")


In [ ]:

# Global variables to store line coordinates
counting_line = None
line_drawn = False
drawing = False
start_point = None
end_point = None
sample_frame = None

def mouse_callback(event, x, y, flags, param):
    """Mouse callback function to draw counting line on sample frame"""
    global counting_line, line_drawn, drawing, start_point, end_point, sample_frame
    
    if event == cv2.EVENT_LBUTTONDOWN:
        start_point = (x, y)
        drawing = True
        print(f"🖱️ Line start: {start_point}")
        
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing and sample_frame is not None:
            temp_frame = sample_frame.copy()
            cv2.line(temp_frame, start_point, (x, y), (0, 255, 255), 3)
            cv2.putText(temp_frame, "Release mouse to set counting line", (50, 80), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 255), 3)
            cv2.imshow('Draw Counting Line - Video Sample', temp_frame)
            
    elif event == cv2.EVENT_LBUTTONUP:
        if drawing:
            end_point = (x, y)
            counting_line = (start_point, end_point)
            line_drawn = True
            drawing = False
            print(f"✅ Line end: {end_point}")
            print(f"✅ Counting line coordinates: {counting_line}")

def setup_line_from_video(video_path):
    """Extract sample frame from video and setup counting line in FULLSCREEN"""
    global sample_frame, counting_line, line_drawn
    
    # Open video to get sample frame
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"❌ Error: Cannot open video file: {video_path}")
        return False
    
    # Read first frame
    ret, frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Error: Cannot read frame from video")
        return False
    
    sample_frame = frame.copy()
    print(f"✅ Sample frame extracted from: {video_path}")
    print(f"📐 Frame size: {frame.shape[1]}x{frame.shape[0]}")
    
    print("📋 Instructions:")
    print("   1. Click and drag on the frame to draw counting line")
    print("   2. Press SPACE to confirm line")
    print("   3. Press 'r' to redraw line")
    print("   4. Press 'q' to cancel")
    print("   5. Press 'f' to toggle fullscreen")
    print("   6. Press ESC to exit fullscreen")
    
    # Create window with fullscreen capability
    cv2.namedWindow('Draw Counting Line - Video Sample', cv2.WINDOW_NORMAL)
    
    # Set to fullscreen mode
    cv2.setWindowProperty('Draw Counting Line - Video Sample', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    cv2.setMouseCallback('Draw Counting Line - Video Sample', mouse_callback)
    
    fullscreen_mode = True
    
    while True:
        display_frame = sample_frame.copy()
        
        # Draw the counting line if exists
        if line_drawn and counting_line:
            cv2.line(display_frame, counting_line[0], counting_line[1], (0, 255, 0), 4)
            cv2.putText(display_frame, "COUNTING LINE SET - Press SPACE to confirm", 
                       (50, display_frame.shape[0] - 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 3)
            
            # Show line coordinates (larger text for fullscreen)
            cv2.putText(display_frame, f"Line: {counting_line[0]} to {counting_line[1]}", 
                       (50, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
        else:
            cv2.putText(display_frame, "Click and drag to draw counting line", 
                       (50, display_frame.shape[0] - 60),
                       cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 0), 3)
        
        # Add control instructions on screen (larger for fullscreen)
        cv2.putText(display_frame, "Controls: SPACE=Confirm | R=Reset | Q=Cancel | F=Toggle Fullscreen", 
                   (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        
        cv2.imshow('Draw Counting Line - Video Sample', display_frame)
        
        key = cv2.waitKey(30) & 0xFF
        if key == ord(' '):  # Space to confirm
            if line_drawn:
                print("✅ Counting line confirmed!")
                break
            else:
                print("⚠️ Please draw a line first!")
        elif key == ord('r'):  # Reset line
            counting_line = None
            line_drawn = False
            print("🔄 Line reset - draw again")
        elif key == ord('q'):  # Quit
            print("❌ Setup cancelled")
            cv2.destroyAllWindows()
            return False
        elif key == ord('f'):  # Toggle fullscreen
            if fullscreen_mode:
                cv2.setWindowProperty('Draw Counting Line - Video Sample', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
                fullscreen_mode = False
                print("🪟 Windowed mode")
            else:
                cv2.setWindowProperty('Draw Counting Line - Video Sample', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
                fullscreen_mode = True
                print("🖥️ Fullscreen mode")
        elif key == 27:  # ESC key to exit fullscreen
            cv2.setWindowProperty('Draw Counting Line - Video Sample', cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_NORMAL)
            fullscreen_mode = False
            print("🪟 Exited fullscreen mode")
    
    cv2.destroyAllWindows()
    return True


In [ ]:

video_path = r'Manufacturing\1.mp4'  # ← VIDEO PATH

print("🎥 Video Sample Frame Setup")
print(f"📁 Video path: {video_path}")
print()
print("To setup counting line:")
print(f"setup_line_from_video('{video_path}')")

# Call the function to setup counting line from video
setup_line_from_video(video_path)

In [ ]:
counting_line

In [ ]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Set your counting line coordinates (start_point, end_point)
counting_line = ((482, 759), (1264, 730))

# Set file paths
video_path = r'Manufacturing\1.mp4'  # ← CHANGE THIS TO YOUR VIDEO PATH
model_path = r'runs\detect\train\weights\last.pt'  # ← CHANGE THIS TO YOUR YOLO MODEL PATH
output_video_path = 'output_counted_video.mp4'  # ← OUTPUT VIDEO PATH

# Tracking settings
max_distance = 100    # Max distance to consider same object
max_disappeared = 30  # Max frames an object can disappear before removal


In [ ]:

# =============================================================================
# GLOBAL VARIABLES
# =============================================================================

product_counter = 0
model = None
object_tracker = {}  # Store tracked objects: {id: {'centers': [], 'bbox': (), 'class_id': int, 'disappeared': int, 'counted': bool}}
next_object_id = 1

def load_yolo_model(model_path):
    """Load YOLO model"""
    global model
    try:
        print(f"📦 Loading YOLO model: {model_path}")
        model = YOLO(model_path)
        print("✅ YOLO model loaded successfully")
        return True
    except Exception as e:
        print(f"❌ Error loading YOLO model: {e}")
        return False

def calculate_distance(point1, point2):
    """Calculate distance between two points"""
    return np.sqrt((point1[0] - point2[0])**2 + (point1[1] - point2[1])**2)

def line_intersection(p1, p2, p3, p4):
    """Check if line p1-p2 intersects with line p3-p4"""
    def ccw(A, B, C):
        return (C[1] - A[1]) * (B[0] - A[0]) > (B[1] - A[1]) * (C[0] - A[0])
    return ccw(p1, p3, p4) != ccw(p2, p3, p4) and ccw(p1, p2, p3) != ccw(p1, p2, p4)

def check_line_crossing(obj_id):
    """Check if object crossed the counting line"""
    global counting_line, product_counter, object_tracker
    
    obj = object_tracker[obj_id]
    if obj['counted'] or len(obj['centers']) < 2:
        return False
    
    # Check if trajectory crosses the counting line
    prev_pos = obj['centers'][-2]
    curr_pos = obj['centers'][-1]
    
    if line_intersection(prev_pos, curr_pos, counting_line[0], counting_line[1]):
        obj['counted'] = True
        product_counter += 1
        print(f"🎯 Object #{product_counter} (ID: {obj_id}) crossed the line!")
        return True
    
    return False

def update_tracker(detections):
    """Update object tracker with new detections"""
    global object_tracker, next_object_id, max_distance, max_disappeared
    
    # Mark all existing objects as potentially disappeared
    for obj in object_tracker.values():
        obj['disappeared'] += 1
    
    # Match detections with existing tracked objects
    for center, bbox, class_id, confidence in detections:
        best_match = None
        best_distance = float('inf')
        
        # Find closest existing object of same class
        for obj_id, obj in object_tracker.items():
            if obj['class_id'] == class_id:
                distance = calculate_distance(center, obj['centers'][-1])
                if distance < max_distance and distance < best_distance:
                    best_distance = distance
                    best_match = obj_id
        
        if best_match is not None:
            # Update existing object
            object_tracker[best_match]['centers'].append(center)
            object_tracker[best_match]['bbox'] = bbox
            object_tracker[best_match]['confidence'] = confidence
            object_tracker[best_match]['disappeared'] = 0
            
            # Keep only last 5 positions
            if len(object_tracker[best_match]['centers']) > 5:
                object_tracker[best_match]['centers'].pop(0)
        else:
            # Create new tracked object
            object_tracker[next_object_id] = {
                'centers': [center],
                'bbox': bbox,
                'class_id': class_id,
                'confidence': confidence,
                'disappeared': 0,
                'counted': False
            }
            next_object_id += 1
    
    # Remove objects that disappeared for too long
    to_remove = [obj_id for obj_id, obj in object_tracker.items() if obj['disappeared'] > max_disappeared]
    for obj_id in to_remove:
        del object_tracker[obj_id]

def process_frame(frame):
    """Process single frame for detection and counting"""
    global model, counting_line, object_tracker
    
    frame_copy = frame.copy()
    
    # Draw counting line
    cv2.line(frame_copy, counting_line[0], counting_line[1], (0, 255, 0), 4)
    mid_x = (counting_line[0][0] + counting_line[1][0]) // 2
    mid_y = (counting_line[0][1] + counting_line[1][1]) // 2
    cv2.putText(frame_copy, "COUNTING LINE", (mid_x - 80, mid_y - 10), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    
    # Run YOLO detection
    detections = []
    try:
        results = model(frame, conf=0.5, verbose=False)
        for result in results:
            if result.boxes is not None:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    confidence = box.conf[0].cpu().numpy()
                    class_id = int(box.cls[0].cpu().numpy())
                    
                    center_x = int((x1 + x2) / 2)
                    center_y = int((y1 + y2) / 2)
                    center_point = (center_x, center_y)
                    bbox = (int(x1), int(y1), int(x2), int(y2))
                    
                    detections.append((center_point, bbox, class_id, confidence))
    except Exception as e:
        print(f"⚠️ Detection error: {e}")
    
    # Update tracker and check crossings
    update_tracker(detections)
    
    for obj_id, obj in object_tracker.items():
        check_line_crossing(obj_id)
        
        # Draw bounding box with color based on status
        x1, y1, x2, y2 = obj['bbox']
        if obj['counted']:
            color = (0, 255, 0)  # Green: counted
            thickness = 3
        else:
            color = (255, 0, 0)  # Blue: not counted
            thickness = 2
        
        cv2.rectangle(frame_copy, (x1, y1), (x2, y2), color, thickness)
        
        # Draw center and ID
        center = obj['centers'][-1]
        cv2.circle(frame_copy, center, 5, (0, 0, 255), -1)
        cv2.putText(frame_copy, f"ID:{obj_id}", (x1, y1 - 10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
        
        # Draw trajectory
        if len(obj['centers']) > 1:
            for i in range(1, len(obj['centers'])):
                cv2.line(frame_copy, obj['centers'][i-1], obj['centers'][i], (255, 255, 0), 2)
    
    # Draw counter
    cv2.rectangle(frame_copy, (10, 10), (200, 60), (0, 0, 0), -1)
    cv2.putText(frame_copy, f"COUNT: {product_counter}", (20, 40),
               cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 2)
    
    return frame_copy

def process_video():
    """Main function to process video"""
    global product_counter, object_tracker, next_object_id
    
    print(f"🎥 Starting Product Counter")
    print(f"📁 Input: {video_path}")
    print(f"📁 Output: {output_video_path}")
    print(f"🤖 Model: {model_path}")
    print("="*50)
    
    # Load model
    if not load_yolo_model(model_path):
        return False
    
    # Open input video
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Cannot open video: {video_path}")
        return False
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"📊 Video: {width}x{height} @ {fps}fps, {total_frames} frames")
    
    # Create output video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))
    
    # Reset counters
    product_counter = 0
    object_tracker = {}
    next_object_id = 1
    
    frame_count = 0
    start_time = datetime.now()
    
    print("🚀 Processing...")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        
        # Process frame
        processed_frame = process_frame(frame)
        out.write(processed_frame)
        
        # Show progress every 10%
        if frame_count % (total_frames // 10) == 0:
            progress = (frame_count / total_frames) * 100
            print(f"📈 {progress:.0f}% - Frame {frame_count}/{total_frames} - Count: {product_counter}")
    
    # Cleanup
    cap.release()
    out.release()
    
    end_time = datetime.now()
    processing_time = end_time - start_time
    
    # Results
    print("="*50)
    print("🏁 Processing completed!")
    print(f"📊 Total count: {product_counter}")
    print(f"📊 Processing time: {processing_time}")
    print(f"📁 Output saved: {output_video_path}")
    
    # Save report
    report_path = output_video_path.replace('.mp4', '_report.txt')
    with open(report_path, 'w') as f:
        f.write(f"Product Counting Report\n")
        f.write(f"======================\n")
        f.write(f"Date: {datetime.now()}\n")
        f.write(f"Input: {video_path}\n")
        f.write(f"Output: {output_video_path}\n")
        f.write(f"Total Count: {product_counter}\n")
        f.write(f"Processing Time: {processing_time}\n")
        f.write(f"Counting Line: {counting_line}\n")
    
    print(f"📄 Report saved: {report_path}")
    return True


In [ ]:
process_video()